<H1> Coursera Capstone Canada Dataset Project

<H4> Import neccessary packages

In [1]:
import pandas as pd
import requests
import lxml.html as lh

<H4> Reading all the tables from webpage and seperate out dataset required

In [2]:
tables = pd.read_html("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M")

In [3]:
canada_df = tables[0]

In [4]:
canada_df.head()

,0,1,2
0,Postcode,Borough,Neighbourhood
1,M1A,Not assigned,Not assigned
2,M2A,Not assigned,Not assigned
3,M3A,North York,Parkwoods
4,M4A,North York,Victoria Village


In [5]:
canada_df.columns = canada_df.iloc[0]
canada_df = canada_df[1:]
canada_df.head()

,Postcode,Borough,Neighbourhood
1,M1A,Not assigned,Not assigned
2,M2A,Not assigned,Not assigned
3,M3A,North York,Parkwoods
4,M4A,North York,Victoria Village
5,M5A,Downtown Toronto,Harbourfront


In [6]:
canada_df["Borough"].describe()

count              288
unique              12
top       Not assigned
freq                77
Name: Borough, dtype: object

<H4> There are 77 rows for which Bororugh is Not assigned. This rows are droped and reset index

In [7]:
canada_df.drop( canada_df[ canada_df["Borough"] == "Not assigned" ].index, inplace = True )   

c:\users\mahen\appdata\local\programs\python\python35\lib\site-packages\pandas\core\frame.py:3697: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  errors=errors)


In [8]:
canada_df["Borough"].describe()

count           211
unique           11
top       Etobicoke
freq             45
Name: Borough, dtype: object

In [9]:
canada_df = canada_df.reset_index(drop = True) 

<H4> The cell with a borough but a Not assigned neighborhood, then the neighborhood is same as the borough

In [10]:
for i in range(len(canada_df)):
    if canada_df.iloc[i,2] == "Not assigned":
        canada_df.iloc[i,2] = canada_df.iloc[i,1]

<H4> The rows with one postal code area but more than one neighborhood are combined

In [11]:
foo = lambda a: " , ".join(a) 

In [12]:
canada_df2 = canada_df.groupby(['Postcode','Borough']).agg({'Neighbourhood': foo}).reset_index()

In [13]:
canada_df2.head()

,Postcode,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge , Malvern"
1,M1C,Scarborough,"Highland Creek , Rouge Hill , Port Union"
2,M1E,Scarborough,"Guildwood , Morningside , West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [14]:
canada_df2.shape

(103, 3)

<H4> Unable to use geocoder so importing csv file

In [15]:
geospa = pd.read_csv('http://cocl.us/Geospatial_data')

In [16]:
geospa.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [17]:
Latitude = pd.Series([])
Longitude = pd.Series([]) 
for i in range(len(canada_df2)):
    for j in range(len(geospa)):
        if canada_df2.iloc[i]['Postcode'] == geospa.iloc[j]['Postal Code']:
            Latitude[i]=geospa.iloc[j]['Latitude']
            Longitude[i]=geospa.iloc[j]['Longitude']


In [18]:
canada_df2.insert(3, "Latitude", Latitude)
canada_df2.insert(4, "Longitude", Longitude)
canada_df2.head()

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge , Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek , Rouge Hill , Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood , Morningside , West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


In [19]:
from sklearn.cluster import KMeans
import folium 

In [23]:
map_canada = folium.Map(location=[43.6532, 79.3832], zoom_start=11)

# set color scheme for the clusters

# add markers to the map

for lat, lon, poi in zip(canada_df2['Latitude'], canada_df2['Longitude'], canada_df2['Neighbourhood']):
    label = folium.Popup(str(poi) , parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        fill=True,
        fill_opacity=0.7).add_to(map_canada)
       
map_canada